# *Impoart Libraries*

In [2]:
import numpy as np
import tensorflow as tf
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# *Assign directories*

In [3]:
# Assign training and validation set directories
base_dir = '/kaggle/input/riceleafs/RiceLeafs/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Assign directories with training pictures 
train_brownspot = os.path.join(train_dir, 'BrownSpot')
train_healthy = os.path.join(train_dir, 'Healthy')
train_hispa = os.path.join(train_dir, 'Hispa')
train_leafblast = os.path.join(train_dir, 'LeafBlast')

# Assign directories with validation pictures
validation_brownspot = os.path.join(validation_dir, 'BrownSpot')
validation_healthy = os.path.join(validation_dir, 'Healthy')
validation_hispa = os.path.join(validation_dir, 'Hispa')
validation_leafblast = os.path.join(validation_dir, 'LeafBlast')

In [4]:
# See the total number of piectures in training/test directories
train_brownspot_files = os.listdir(train_brownspot)
train_healthy_files = os.listdir(train_healthy)
train_hispa_files = os.listdir(train_hispa)
train_leafblast_files = os.listdir(train_leafblast)

validation_brownspot_files = os.listdir(validation_brownspot)
validation_healthy_files = os.listdir(validation_healthy)
validation_hispa_files = os.listdir(validation_hispa)
validation_leafblast_files = os.listdir(validation_leafblast)

brownspot_counts = len(train_brownspot_files)
healthy_counts = len(train_healthy_files)
hispa_counts = len(train_hispa_files)
leafblast_counts = len(train_leafblast_files)

print('-Total training images- \n',
      'BrownSpot: ', brownspot_counts, '\n',
      'Healthy: ', healthy_counts, '\n',
      'Hispa: ', hispa_counts, '\n',
      'LeafBlast: ', leafblast_counts, '\n')

print('-Total validation images- \n',
      'BrownSpot: ', len(validation_brownspot_files), '\n',
      'Healthy: ', len(validation_healthy_files), '\n',
      'Hispa: ', len(validation_hispa_files), '\n',
      'LeafBlast: ', len(validation_leafblast_files))

-Total training images- 
 BrownSpot:  418 
 Healthy:  1191 
 Hispa:  452 
 LeafBlast:  623 

-Total validation images- 
 BrownSpot:  105 
 Healthy:  297 
 Hispa:  113 
 LeafBlast:  156


In [5]:
# Check the filename of training/validation files
print('-File name of training images- \n',
      'BrownSpot: \n', train_brownspot_files[:5], '\n',
      'Healthy: \n', train_healthy_files[:5], '\n',
      'Hispa: \n', train_hispa_files[:5], '\n',
      'LeafBlast: \n', train_leafblast_files[:5], '\n')
print('-File name of validation images- \n',
      'BrownSpot: \n', validation_brownspot_files[:5], '\n',
      'Healthy: \n', validation_healthy_files[:5], '\n',
      'Hispa: \n', validation_hispa_files[:5], '\n',
      'LeafBlast: \n', validation_leafblast_files[:5])

-File name of training images- 
 BrownSpot: 
 ['IMG_20190420_193445.jpg', 'IMG_20190420_194208.jpg', 'IMG_20190420_193850.jpg', 'IMG_20190424_132317.jpg', 'IMG_20190421_195708.jpg'] 
 Healthy: 
 ['IMG_20190419_170344.jpg', 'IMG_20190419_134548.jpg', 'IMG_20190424_133839.jpg', 'IMG_20190419_164031.jpg', 'IMG_20190419_173858.jpg'] 
 Hispa: 
 ['IMG_20190419_144654.jpg', 'IMG_20190419_144252.jpg', 'IMG_20190419_161439.jpg', 'IMG_20190420_200351.jpg', 'IMG_20190424_133939.jpg'] 
 LeafBlast: 
 ['IMG_20190419_112054.jpg', 'IMG_20190419_134955.jpg', 'IMG_20190419_174405.jpg', 'IMG_20190419_162730.jpg', 'IMG_20190420_194944.jpg'] 

-File name of validation images- 
 BrownSpot: 
 ['IMG_20190419_103257.jpg', 'IMG_20190419_113242.jpg', 'IMG_20190420_185515.jpg', 'IMG_20190419_145246.jpg', 'IMG_20190419_113228.jpg'] 
 Healthy: 
 ['IMG_20190419_100514.jpg', 'IMG_20190419_095521.jpg', 'IMG_20190419_100701.jpg', 'IMG_3030.jpg', 'IMG_20190419_103358.jpg'] 
 Hispa: 
 ['IMG_20190419_103319.jpg', 'IMG_201

# *Check a random batch of images of each category*

In [6]:
# Show random 9 images from specific category.
def plot_images(category):
    category_path = globals()['train_' + category]
    filename = globals()['train_' + category + '_files']

    random_images = random.sample(filename, 9)

    nrows = 3
    ncols = 3
    fig, axes = plt.subplots(nrows, ncols, figsize = (6, 6))
    
    for i, img_file in enumerate(random_images):
        img_path = os.path.join(category_path, img_file) 
        img = mpimg.imread(img_path)
        sp = plt.subplot(nrows, ncols, i+1)
        sp.axis('off')
        plt.imshow(img)
        
    fig.suptitle(category + ' images')    
    plt.tight_layout()
    plt.show()

In [7]:
# plot_images('brownspot')

In [8]:
# plot_images('healthy')

In [9]:
# plot_images('hispa')

In [10]:
# plot_images('leafblast')

# *Data preprocessing*

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
class_counts = [brownspot_counts, healthy_counts, hispa_counts, leafblast_counts]
total_samples = sum(class_counts)
class_weights = [total_samples / (4 * count) for count in class_counts]
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

Class Weights: {0: 1.605263157894737, 1: 0.563392107472712, 2: 1.4845132743362832, 3: 1.0770465489566614}


In [27]:
img_size = 200
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 2684 images belonging to 4 classes.
Found 671 images belonging to 4 classes.


# *Model building*

In [28]:
# #Try 1
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
# #     tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
# #     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(4, activation='softmax')

# ])

# model.summary()

In [29]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-06-22 08:00:11--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 108.177.12.128, 108.177.13.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   137MB/s    in 0.6s    

2023-06-22 08:00:12 (137 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [30]:
# Transfer learning InceptionV3
from tensorflow.keras.applications import InceptionV3


pre_trained_model = InceptionV3(weights=None,
                                include_top=False,
                                input_shape=(img_size, img_size, 3))

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 99, 99, 32)   864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (None, 99, 99, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                       

In [ ]:
last_layer = pre_trained_model.get_layer('mixed6')
print(last_layer.output_shape)
last_output = last_layer.output

In [44]:
from tensorflow.keras import regularizers

# x = layers.GlobalAveragePooling2D()(last_output)
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation='relu')(x)
# x = layers.Dropout(0.3)(x)
x = layers.Dense(4, activation='softmax')(x)
model = Model(pre_trained_model.input, x)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 99, 99, 32)   864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (None, 99, 99, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [26]:
# from tensorflow.keras.applications import InceptionV3


# base_model = InceptionV3(weights='imagenet',
#                                 include_top=False,
#                                 input_shape=(img_size, img_size, 3))


# model = tf.keras.models.Sequential([
#     base_model,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
# #     tf.keras.layers.Dense(64, activation='relu'),
# #     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(4, activation='softmax')
# ])

# model.summary()

# base_model.trainable = False

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 4, 4, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                      

In [20]:
# from tensorflow.keras.applications import ResNet50
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# model = tf.keras.models.Sequential([
#     base_model,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(4, activation='softmax')
# ])

# model.summary()

# base_model.trainable = False

In [45]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5)

model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics = ['accuracy'])

# *Train the model*

In [46]:
EPOCHS = 30
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//validation_generator.batch_size,
                    class_weight=class_weights_dict,
                    callbacks=[lr_scheduler],
                    verbose=1)

Epoch 1/30
149/167 [=========================>....] - ETA: 32s - loss: 2.5182 - accuracy: 0.3185


KeyboardInterrupt



In [ ]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', lalbel='Validation accuracy')
# plt.legend(loc=0)
# plt.show

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()